# 1. Get the previous data

In [31]:
import requests
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Get Wikipedia page is converted to a string
raw_random_wikipedia_page=requests.get(wikipedia_link)
#extract the XML as a text file a string and assign the result variable page
page = raw_random_wikipedia_page.text
# Extracting the postal codes into dataframe
import pandas as pd
from bs4 import BeautifulSoup as bs
html_soup = bs(page,'html.parser')
table_rows = html_soup.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        if str(row).startswith("['NL") or str(row).startswith("['A"):
            res
        else:
            res.append(row)
        
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])
#df_sorted = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])
df_sorted = df.sort_values(by='PostalCode').reset_index()
del df_sorted['index']

df_sorted = df_sorted.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

print(df_sorted.loc[(df_sorted['Borough'] != 'Not assigned') & (df_sorted['Neighbourhood'] == 'Not assigned')])   

for index, row in df_sorted.iterrows():
    
    if (df_sorted.loc[index,'Borough'] != 'Not assigned') & (df_sorted.loc[index,'Neighbourhood'] == 'Not assigned'):
        Borough = df_sorted.loc[index,'Borough']
        df_sorted.loc[index,'Neighbourhood'] = Borough
                      
print (df_sorted.head(10))

    PostalCode       Borough Neighbourhood
120        M7A  Queen's Park  Not assigned
  PostalCode       Borough                                    Neighbourhood
0        M1A  Not assigned                                     Not assigned
1        M1B   Scarborough                                   Malvern, Rouge
2        M1C   Scarborough           Port Union, Rouge Hill, Highland Creek
3        M1E   Scarborough                West Hill, Morningside, Guildwood
4        M1G   Scarborough                                           Woburn
5        M1H   Scarborough                                        Cedarbrae
6        M1J   Scarborough                              Scarborough Village
7        M1K   Scarborough      East Birchmount Park, Ionview, Kennedy Park
8        M1L   Scarborough                  Oakridge, Golden Mile, Clairlea
9        M1M   Scarborough  Cliffcrest, Cliffside, Scarborough Village West


# 2.  get the geographical coordinates of the neighborhoods in the Toronto
*imported the CSV into dataframe*

In [32]:
website_URL = "https://cocl.us/Geospatial_data"
    
df_geo = pd.read_csv(website_URL)
print(df_geo.head(5))

df_geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
print(df_geo.head(5))

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476


In [33]:
df_postal = df_sorted.merge(df_geo, on="PostalCode", how = 'inner')
print (df_postal.head(10))

  PostalCode      Borough                                    Neighbourhood  \
0        M1B  Scarborough                                   Malvern, Rouge   
1        M1C  Scarborough           Port Union, Rouge Hill, Highland Creek   
2        M1E  Scarborough                West Hill, Morningside, Guildwood   
3        M1G  Scarborough                                           Woburn   
4        M1H  Scarborough                                        Cedarbrae   
5        M1J  Scarborough                              Scarborough Village   
6        M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park   
7        M1L  Scarborough                  Oakridge, Golden Mile, Clairlea   
8        M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West   
9        M1N  Scarborough                      Cliffside West, Birch Cliff   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.77313